# GARCH(1,1) Volatility Derivation: Complete Mathematical Proof

## Problem Statement

**Given:** A time series of battery capacity measurements from NASA Li-ion Battery B0047

**Find:** The GARCH(1,1) model parameters that characterize variance dynamics

**Model:**
$$\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2$$

where:
- $\omega$ = baseline variance (long-run)
- $\alpha$ = shock impact coefficient
- $\beta$ = variance persistence
- $\epsilon_t$ = innovation (shock) at time t

---

## Step 0: Load the Data

In [ ]:
import numpy as np
import polars as pl
np.set_printoptions(precision=6, suppress=True)

# Load battery data
obs = pl.read_parquet('../../data/battery_45_48/observations.parquet')
b47 = obs.filter(
    (pl.col('entity_id') == 'B0047') & 
    (pl.col('signal_id') == 'capacity')
).sort('timestamp')

X = b47['value'].to_numpy()
n = len(X)

print(f"GARCH(1,1) Model Estimation")
print(f"="*60)
print(f"\nTotal observations: n = {n}")
print(f"\nFirst 15 capacity values (Ah):")
print(f"\n  {'t':<4} {'xt (Ah)':<12}")
print(f"  {'-'*4} {'-'*12}")
for i in range(15):
    print(f"  {i:<4} {X[i]:.6f}")

---

## Step 1: Compute Returns (First Differences)

### Definition:
GARCH models variance of returns/changes, not levels:
$$r_t = x_t - x_{t-1}$$

### Solution:

In [ ]:
print("Step 1: Compute Returns (First Differences)")
print("="*60)
print(f"\nFormula: rₜ = xₜ - xₜ₋₁")

# Compute returns
r = np.diff(X)  # r[0] = X[1] - X[0], etc.
T = len(r)  # Number of returns

print(f"\nNumber of returns: T = n - 1 = {n} - 1 = {T}")
print(f"\nComputing first 10 returns:")
print(f"\n  {'t':<4} {'xₜ':<12} {'-':<3} {'xₜ₋₁':<12} {'=':<3} {'rₜ':<12}")
print(f"  {'-'*4} {'-'*12} {'-'*3} {'-'*12} {'-'*3} {'-'*12}")
for t in range(10):
    print(f"  {t+1:<4} {X[t+1]:<12.6f} {'-':<3} {X[t]:<12.6f} {'=':<3} {r[t]:+.6f}")

print(f"\nReturn statistics:")
print(f"  Mean(r): μ = {np.mean(r):.6f}")
print(f"  Std(r):  σ = {np.std(r):.6f}")
print(f"  Min(r):     {np.min(r):.6f}")
print(f"  Max(r):     {np.max(r):.6f}")

---

## Step 2: De-mean the Returns

### Definition:
The GARCH model assumes mean-zero innovations:
$$\epsilon_t = r_t - \mu$$

where $\mu = \frac{1}{T}\sum_{t=1}^{T} r_t$

### Solution:

In [ ]:
print("Step 2: De-mean the Returns (Compute Innovations)")
print("="*60)
print(f"\nFormula: εₜ = rₜ - μ")

# Compute mean
mu = np.mean(r)
print(f"\nMean of returns:")
print(f"  μ = (1/T) × Σrₜ")
print(f"    = (1/{T}) × {np.sum(r):.6f}")
print(f"    = {mu:.6f}")

# De-mean
epsilon = r - mu

print(f"\nComputing innovations (first 10):")
print(f"\n  {'t':<4} {'rₜ':<12} {'-':<3} {'μ':<12} {'=':<3} {'εₜ':<12}")
print(f"  {'-'*4} {'-'*12} {'-'*3} {'-'*12} {'-'*3} {'-'*12}")
for t in range(10):
    print(f"  {t+1:<4} {r[t]:+.6f}    {'-':<3} {mu:.6f}     {'=':<3} {epsilon[t]:+.6f}")

print(f"\nVerification: Mean(ε) = {np.mean(epsilon):.10f} ≈ 0 ✓")

---

## Step 3: Compute Squared Innovations

### Definition:
GARCH models the dynamics of squared innovations (variance proxy):
$$\epsilon_t^2$$

### Solution:

In [ ]:
print("Step 3: Compute Squared Innovations")
print("="*60)
print(f"\nFormula: εₜ² (proxy for instantaneous variance)")

# Squared innovations
epsilon_sq = epsilon ** 2

print(f"\nComputing squared innovations (first 15):")
print(f"\n  {'t':<4} {'εₜ':<14} {'εₜ²':<14}")
print(f"  {'-'*4} {'-'*14} {'-'*14}")
for t in range(15):
    print(f"  {t+1:<4} {epsilon[t]:+.6f}      {epsilon_sq[t]:.8f}")

print(f"\nSquared innovation statistics:")
print(f"  Mean(ε²):    {np.mean(epsilon_sq):.8f}")
print(f"  Variance(ε): {np.var(epsilon):.8f}")
print(f"  Max(ε²):     {np.max(epsilon_sq):.8f}")

---

## Step 4: GARCH(1,1) Model Specification

### Model:
$$\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2$$

### Constraints:
- $\omega > 0$ (positive baseline variance)
- $\alpha \geq 0$ (non-negative shock impact)
- $\beta \geq 0$ (non-negative persistence)
- $\alpha + \beta < 1$ (stationarity condition)

### Unconditional Variance:
$$\bar{\sigma}^2 = \frac{\omega}{1 - \alpha - \beta}$$

In [ ]:
print("Step 4: GARCH(1,1) Model Specification")
print("="*60)
print(f"""
Model: σₜ² = ω + α·εₜ₋₁² + β·σₜ₋₁²

Parameters to estimate:
  ω (omega): Baseline variance component
  α (alpha): Shock impact coefficient  
  β (beta):  Variance persistence coefficient

Constraints:
  ω > 0
  α ≥ 0
  β ≥ 0  
  α + β < 1 (stationarity)

Key quantities:
  Persistence = α + β
  Unconditional variance = ω / (1 - α - β)
""")

---

## Step 5: Maximum Likelihood Estimation Setup

### Log-Likelihood Function:
Assuming Gaussian innovations:
$$\mathcal{L}(\omega, \alpha, \beta) = -\frac{T}{2}\ln(2\pi) - \frac{1}{2}\sum_{t=1}^{T}\left[\ln(\sigma_t^2) + \frac{\epsilon_t^2}{\sigma_t^2}\right]$$

### Solution:

In [ ]:
print("Step 5: Maximum Likelihood Estimation")
print("="*60)
print(f"""
Log-Likelihood (Gaussian):

  L(ω,α,β) = -(T/2)·ln(2π) - (1/2)·Σ[ln(σₜ²) + εₜ²/σₜ²]

We maximize this by iterating over possible parameter values.
""")

def garch_variance(epsilon_sq, omega, alpha, beta):
    """Compute GARCH(1,1) conditional variances."""
    T = len(epsilon_sq)
    sigma2 = np.zeros(T)
    
    # Initialize with unconditional variance
    sigma2[0] = np.var(epsilon_sq[:10]) if omega / (1 - alpha - beta) <= 0 else omega / (1 - alpha - beta)
    
    # Recursion
    for t in range(1, T):
        sigma2[t] = omega + alpha * epsilon_sq[t-1] + beta * sigma2[t-1]
    
    return sigma2

def garch_log_likelihood(params, epsilon_sq):
    """Compute negative log-likelihood for GARCH(1,1)."""
    omega, alpha, beta = params
    
    # Constraints
    if omega <= 0 or alpha < 0 or beta < 0 or alpha + beta >= 1:
        return 1e10
    
    T = len(epsilon_sq)
    sigma2 = garch_variance(epsilon_sq, omega, alpha, beta)
    
    # Avoid numerical issues
    sigma2 = np.maximum(sigma2, 1e-10)
    
    # Log-likelihood
    ll = -0.5 * np.sum(np.log(sigma2) + epsilon_sq / sigma2)
    
    return -ll  # Return negative for minimization

print("Log-likelihood function defined.")

---

## Step 6: Grid Search for Initial Parameters

In [ ]:
print("Step 6: Grid Search for Parameter Estimation")
print("="*60)

# Sample variance as starting point
sample_var = np.var(epsilon)
print(f"\nSample variance of innovations: σ² = {sample_var:.8f}")

# Grid search
best_ll = -np.inf
best_params = None

print(f"\nSearching parameter space:")
print(f"  ω ∈ [0.00001, 0.001]")
print(f"  α ∈ [0.01, 0.3]")
print(f"  β ∈ [0.5, 0.95]")

omega_grid = np.linspace(0.00001, 0.001, 20)
alpha_grid = np.linspace(0.01, 0.3, 15)
beta_grid = np.linspace(0.5, 0.95, 15)

results = []
for omega in omega_grid:
    for alpha in alpha_grid:
        for beta in beta_grid:
            if alpha + beta < 1:
                neg_ll = garch_log_likelihood([omega, alpha, beta], epsilon_sq)
                ll = -neg_ll
                results.append((omega, alpha, beta, ll))
                if ll > best_ll:
                    best_ll = ll
                    best_params = (omega, alpha, beta)

omega_opt, alpha_opt, beta_opt = best_params
persistence = alpha_opt + beta_opt
unconditional_var = omega_opt / (1 - persistence)

print(f"\nGrid search complete ({len(results)} parameter combinations tested)")
print(f"\n" + "-"*60)
print(f"Best parameters found:")
print(f"\n  ω (omega) = {omega_opt:.8f}")
print(f"  α (alpha) = {alpha_opt:.6f}")
print(f"  β (beta)  = {beta_opt:.6f}")
print(f"\n  Persistence = α + β = {alpha_opt:.6f} + {beta_opt:.6f} = {persistence:.6f}")
print(f"  Unconditional variance = ω/(1-α-β) = {unconditional_var:.8f}")
print(f"\n  Log-likelihood = {best_ll:.4f}")

---

## Step 7: Detailed Variance Recursion

### Formula:
$$\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2$$

### Solution with actual values:

In [ ]:
print("Step 7: Detailed Variance Recursion")
print("="*60)
print(f"\nUsing estimated parameters:")
print(f"  ω = {omega_opt:.8f}")
print(f"  α = {alpha_opt:.6f}")
print(f"  β = {beta_opt:.6f}")

print(f"\nFormula: σₜ² = ω + α·εₜ₋₁² + β·σₜ₋₁²")
print(f"\nStep-by-step variance computation:")

# Compute variances
sigma2 = garch_variance(epsilon_sq, omega_opt, alpha_opt, beta_opt)

print(f"\nInitialization:")
print(f"  σ₀² = unconditional variance = ω/(1-α-β) = {sigma2[0]:.8f}")

print(f"\n" + "-"*60)
print(f"Recursion (first 10 steps):")
print(f"\n  {'t':<4} {'Formula':<60} {'σₜ²':<14}")
print(f"  {'-'*4} {'-'*60} {'-'*14}")

for t in range(1, min(11, T)):
    term1 = omega_opt
    term2 = alpha_opt * epsilon_sq[t-1]
    term3 = beta_opt * sigma2[t-1]
    
    formula = f"{omega_opt:.6f} + {alpha_opt:.4f}×{epsilon_sq[t-1]:.6f} + {beta_opt:.4f}×{sigma2[t-1]:.6f}"
    print(f"  {t:<4} {formula:<60} {sigma2[t]:.8f}")

print(f"\n" + "-"*60)
print(f"\nExpanded calculation for t=2:")
t = 2
print(f"\n  σ₂² = ω + α·ε₁² + β·σ₁²")
print(f"\n      = {omega_opt:.8f}")
print(f"        + {alpha_opt:.6f} × {epsilon_sq[t-1]:.8f}")
print(f"        + {beta_opt:.6f} × {sigma2[t-1]:.8f}")
print(f"\n      = {omega_opt:.8f}")
print(f"        + {alpha_opt * epsilon_sq[t-1]:.8f}")
print(f"        + {beta_opt * sigma2[t-1]:.8f}")
print(f"\n      = {sigma2[t]:.8f}")

---

## Step 8: Compute Conditional Volatility

### Definition:
$$\sigma_t = \sqrt{\sigma_t^2}$$

In [ ]:
print("Step 8: Conditional Volatility (Standard Deviation)")
print("="*60)
print(f"\nFormula: σₜ = √σₜ²")

sigma = np.sqrt(sigma2)

print(f"\nConditional volatility (first 15 periods):")
print(f"\n  {'t':<4} {'σₜ²':<14} {'σₜ':<14}")
print(f"  {'-'*4} {'-'*14} {'-'*14}")
for t in range(15):
    print(f"  {t+1:<4} {sigma2[t]:.8f}     {sigma[t]:.8f}")

print(f"\nVolatility statistics:")
print(f"  Mean(σ):  {np.mean(sigma):.6f}")
print(f"  Min(σ):   {np.min(sigma):.6f}")
print(f"  Max(σ):   {np.max(sigma):.6f}")

---

## Step 9: Interpretation of Parameters

### Key Metrics:

| Parameter | Meaning |
|-----------|--------|
| ω (omega) | Baseline variance contribution |
| α (alpha) | Impact of recent shock on variance |
| β (beta) | Persistence of past variance |
| α + β | Total persistence (half-life of shocks) |

In [ ]:
print("Step 9: Physical Interpretation")
print("="*60)

print(f"\nEstimated GARCH(1,1) Parameters:")
print(f"\n  ω (omega) = {omega_opt:.8f}")
print(f"  α (alpha) = {alpha_opt:.6f}")
print(f"  β (beta)  = {beta_opt:.6f}")

print(f"\n" + "-"*60)
print(f"Interpretation:")

# Persistence
print(f"\n1. PERSISTENCE = α + β = {persistence:.6f}")
if persistence > 0.95:
    print(f"   → Very high persistence (>0.95)")
    print(f"   → Variance shocks decay very slowly")
    print(f"   → Near-integrated GARCH (IGARCH)")
elif persistence > 0.85:
    print(f"   → High persistence (0.85-0.95)")
    print(f"   → Variance shocks persist for many periods")
elif persistence > 0.5:
    print(f"   → Moderate persistence (0.5-0.85)")
    print(f"   → Variance shocks decay at moderate rate")
else:
    print(f"   → Low persistence (<0.5)")
    print(f"   → Variance shocks decay quickly")

# Half-life
if persistence > 0 and persistence < 1:
    half_life = np.log(0.5) / np.log(persistence)
    print(f"\n   Half-life of variance shocks: {half_life:.1f} periods")

# Alpha interpretation
print(f"\n2. α (ARCH effect) = {alpha_opt:.6f}")
print(f"   → {alpha_opt*100:.1f}% of variance comes from most recent shock")

# Beta interpretation  
print(f"\n3. β (GARCH effect) = {beta_opt:.6f}")
print(f"   → {beta_opt*100:.1f}% of variance inherited from previous period")

# Unconditional variance
print(f"\n4. Unconditional variance = {unconditional_var:.8f}")
print(f"   Unconditional volatility = {np.sqrt(unconditional_var):.6f}")

print(f"\n" + "-"*60)
print(f"For Battery B0047 capacity:")
print(f"  • {'High' if persistence > 0.85 else 'Moderate'} variance persistence")
print(f"  • Capacity changes show {'stable' if persistence > 0.85 else 'moderate'} volatility dynamics")
print(f"  • Shocks to variance decay over ~{half_life:.0f} cycles" if persistence < 1 else "")

---

## Summary

### Complete Solution

In [ ]:
print("="*70)
print("COMPLETE SOLUTION SUMMARY")
print("="*70)

half_life_str = f"{half_life:.1f}" if persistence < 1 else "∞"

print(f"""
GIVEN:
  • Battery B0047 capacity time series
  • n = {n} observations, T = {T} returns
  • Range: [{X.min():.4f}, {X.max():.4f}] Ah

MODEL: GARCH(1,1)
  σₜ² = ω + α·εₜ₋₁² + β·σₜ₋₁²

STEPS:
  1. Compute returns: rₜ = xₜ - xₜ₋₁
     Mean return: μ = {mu:.6f}
  
  2. Compute innovations: εₜ = rₜ - μ
     Sample variance: {sample_var:.8f}
  
  3. Maximum Likelihood Estimation:
     Maximize: L = -(T/2)·ln(2π) - (1/2)·Σ[ln(σₜ²) + εₜ²/σₜ²]
  
  4. Variance recursion with optimal parameters:
     σₜ² = {omega_opt:.8f} + {alpha_opt:.6f}·εₜ₋₁² + {beta_opt:.6f}·σₜ₋₁²

DETAILED CALCULATION (t=2):
  σ₂² = ω + α·ε₁² + β·σ₁²
      = {omega_opt:.8f} + {alpha_opt:.6f}×{epsilon_sq[1]:.8f} + {beta_opt:.6f}×{sigma2[1]:.8f}
      = {sigma2[2]:.8f}

╔════════════════════════════════════════════════════════════════╗
║                                                                ║
║   GARCH(1,1) PARAMETERS:                                       ║
║                                                                ║
║     ω (omega) = {omega_opt:.8f}                               ║
║     α (alpha) = {alpha_opt:.6f}                                 ║
║     β (beta)  = {beta_opt:.6f}                                 ║
║                                                                ║
║   PERSISTENCE = α + β = {persistence:.6f}                       ║
║   HALF-LIFE = {half_life_str} periods                                   ║
║                                                                ║
║   INTERPRETATION: {'High' if persistence > 0.85 else 'Moderate'} variance persistence             ║
║   Battery capacity volatility is {'stable' if persistence > 0.85 else 'moderately dynamic'}              ║
║                                                                ║
╚════════════════════════════════════════════════════════════════╝
""")

---

*PRISM Behavioral Geometry Engine - Mathematical Derivation Proof*

*Battery: NASA B0047 | Signal: Capacity (Ah) | Method: GARCH(1,1) Volatility Model*